### Import Libraries

In [ ]:
import os
import torch
import numpy as np
import torchio as tio
import SimpleITK as sitk

from tqdm import tqdm
from torchio.transforms import (Compose,
                                ToCanonical,
                                RescaleIntensity, 
                                CropOrPad)

### Download Raw Data

In [ ]:
DATA_PATH = '../data/raw/'

'''Transformation'''
ixi_dataset = tio.datasets.IXI(DATA_PATH,
                               modalities=('T1','T2','PD',),
                               download=True)

### Process Data

In [ ]:
"""Load Data"""
directory = '../data/raw/'
os.mkdir('../data/processed_IXI/')
os.mkdir('../data/processed_IXI/T1')
os.mkdir('../data/processed_IXI/T2')
os.mkdir('../data/processed_IXI/PD')

t2_paths = []
subjects_name =[]
for d in np.sort(os.listdir(directory+'T2/')):
    if 'IXI580-IOP-1157' not in directory+'T2/'+d:
        subjects_name += [d[:-10]] 
        t2_paths += [directory+'T2/'+d]

t1_paths = []
for d in subjects_name:
    if not os.path.exists(directory+'T1'+'/'+d+'-T1.nii.gz'):
        raise Exception('PATH does not exist')
    t1_paths += [directory+'T1'+'/'+d+'-T1.nii.gz']

pd_paths = []
for d in subjects_name: 
    if not os.path.exists(directory+'PD'+'/'+d+'-PD.nii.gz'):
        raise Exception('PATH does not exist')    
    pd_paths += [directory+'PD'+'/'+d+'-PD.nii.gz']

    
'''Load'''
_mod = ['T2','T1','PD']
for j, paths in enumerate([t2_paths, t1_paths, pd_paths]):
    subjects = []
    for i in range(len(paths)):
        subject = tio.Subject(mri= tio.ScalarImage(paths[i]),
                              name=subjects_name[i])
        subjects.append(subject)

    '''Transformation'''
    intensity_norm = RescaleIntensity((0,1),
                                      (0.5, 99.5))

    transforms = Compose([tio.Resample(1),
                          intensity_norm,
                          ToCanonical(1), 
                          CropOrPad(target_shape=(256,256,256))])

    train_set = tio.SubjectsDataset(subjects, 
                                    transform=transforms)

    train_loader = torch.utils.data.DataLoader(train_set,
                                               batch_size=1,
                                               shuffle=False, 
                                               drop_last=False, 
                                               num_workers=0)
    
    for i,data in tqdm(enumerate(train_loader)):
        x = data['mri'][tio.DATA].squeeze().numpy()
        name = data['name'][0]
        x = sitk.GetImageFromArray(x)
        sitk.WriteImage(x, '../data/processed_IXI/{}/{}.nii'.format(_mod[j], name))